# Homework 4

## Graphical Parameters and Model Structure

In the previous homework, you performed queries on a graphical model of possible murders and murder weapons. Now, you will estimate model parameters and structure using data.   

As a reminder, the joint probability distribution is:

$$p(B,C,BW,CW,M)$$     

where the letters indicate the following variables;   
$B = $ butler committed the crime, {not murderer, murderer},   
$C = $ cook committed the crime, {not murderer, murderer},    
$BW = $ choice of weapon, {knife, knife with poison}, conditional on butler,  
$CW = $ choice of weapon, {knife, knife with poison}, conditional on cook,   
$M = $ murderer {butler or cook, third party alone, combination of butler or cook and third party}.    

Keeping in mind it is possible the cook, the butler and the third party could be guilty, and that participation in the crime in any capacity constitutes guilt, the distribution can be factorized in the following manner:

$$p(B,C,BW,CW,M) = p(B)\ p(C)\ p(BW\ |\ B)\ p(CW\ |\ C)\ p(M\ |\ BW,CW, C, W)$$  

A graph of the model is shown below. 

<img src="MurderDirected.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center> **DAG for murder evidence** </center>

Notice that the skeleton of this graph does not have a tree structure. This fact will limit how well estimation algorithms will work, particularly for graph structure. Keep this fact in mind as you proceed. 

As a first step execute the code in the below to simulate the 20 cases for the dataset. Examine the code to see the CPD tables for this simulation. 

In [ ]:
## Simulate the binary tables
import numpy as np
import numpy.random as nr
import pandas as pd

def sim_bernoulli(p, n = 25):
    """
    Function to compute the vectors with probabilities for each 
    condition (input value) of the dependent variable using the Bernoulli
    distribution. 
    
    The arguments are:
    p - a vector of probabilites of success for each case.
    n - The numer of realizations. 
    """
    temp = np.zeros(shape = (len(p), n))
    for i in range(len(p)): 
        temp[i,:] = nr.binomial(1, p[i], n)
    return(temp)

def selec_dist_1(sims, var, lg):
    """
    Function to integrate the conditional probabilities for
    each of the cases of the parent variable. 
    
    The arguments are:
    sims - the array of simulated realizations with one row for each state of the
           parent variable. 
    var - the vector of values of parent variable used to select the value from the 
          sims array.
    lg - vector of states of possible states of the parent variable. These must be
         in the same order as for the sims array. 
    """
    out = sims[0,:] # Copy of values for first parent state
    var = np.array(var).ravel()
    for i in range(1, sims.shape[0]): # loop over other parent states
        out = [x if u == lg[i] else y for x,y,u in zip(sims[i,:], out, var)]
    return([int(x) for x in out])

def set_class_2(x):
    """
    Function to flatten the array produced by the numpy.random.multinoulli function. 
    The function tests which binary value of the array of output states is true
    and substitutes an integer for that state. This function only works for up to three
    output states. 
    
    Argument:
    x - The array produced by the numpy.random.multinoulli function. 
    """
    out = []
    for i,j in enumerate(x):
        if j[0] == 1: out.append(0)
        elif j[1] == 1: out.append(1)
        else: out.append(2)       
    return(out)   

def sim_multinoulli(p, n = 25):
    """
    Function to compute the vectors with probabilities for each 
    condition (input value) of the dependent variable using the multinoulli
    distribution. 
    
    The arguments are:
    p - an array of probabilites of success for each possible combination
        of states of the parent variables. Each row in the array are the 
        probabilities for each state of the multinoulli distribution for 
        that combination of parent values.
    n - The numer of realizations. 
    """
    temp = np.zeros(shape = (p.shape[0], n))
    for i in range(p.shape[0]):  
        ps = p[i,:]
        mutlis = nr.multinomial(1, ps, n) 
        temp[i,:] = set_class_2(mutlis)
    return(temp)

def selec_dist_4(sims, var1, var2, var3, var4, lg1, lg2, lg3, lg4):
    """
    Function to integrate the conditional probabilities for
    each of the cases of four parent variables. 
    
    The arguments are:
    sims - the array of simulated realizations with one row for each state of the
           union of the parent variables. 
    var1 - the vector of values of first parent variable used to select the value from the 
          sims array.
    var2 - the vector of values of second parent variable used to select the value from the 
          sims array.
    var3 - the vector of values of third parent variable used to select the value from the 
          sims array.
    var4 - the vector of values of fourth parent variable used to select the value from the 
          sims array.
    lg1 - vector of states of possible states of the first parent variable. These must be
         in the same order as for the sims array. 
    lg2 - vector of states of possible states of the second parent variable. These must be
         in the same order as for the sims array. 
    lg3 - vector of states of possible states of the third parent variable. These must be
         in the same order as for the sims array. 
    lg4 - vector of states of possible states of the fourth parent variable. These must be
         in the same order as for the sims array. 
    """
    out = sims[0,:] # Copy values for first combination of states for parent variables
    
    ## make sure the parent variables are 1-d numpy arrays.
    var1 = np.array(var1).ravel() 
    var2 = np.array(var2).ravel()
    var3 = np.array(var3).ravel() 
    var4 = np.array(var4).ravel()
    for i in range(1, sims.shape[0]): # Loop over all combination of states of the parent variables
        out = [x if t == lg1[i] and u == lg2[i] and v == lg3[i] and w == lg4[i] else y for x,y,t,u,v,w in 
              zip(sims[i,:], out, var1, var2, var3, var4)]                         
    return([int(x) for x in out])


## set the sample size
nsamp = 20

## First the conditionally independent variables
nr.seed(22234)
B_samps = pd.DataFrame(nr.binomial(1, 0.8, nsamp), columns = ['B'])
nr.seed(2355)
C_samps = pd.DataFrame(nr.binomial(1, 0.3, nsamp), columns = ['C'])

## Two variables conditionally depenent on one other
probs = [0.5, 0.1]
nr.seed(2134)
bern = sim_bernoulli(probs, nsamp)
BW_samps = pd.DataFrame(selec_dist_1(bern, B_samps, [0,1]), columns = ['BW'])

probs = [0.5, 0.7]
nr.seed(22234)
bern = sim_bernoulli(probs)
CW_samps = pd.DataFrame(selec_dist_1(bern, C_samps, [0,1]), columns = ['CW'])


probs = np.array([[0.0, 0.7, 0.5,  0.7, 0.0, 0.6, 0.5,  0.6, 0.0, 0.7, 0.5,  0.7, 0.0, 0.6, 0.5,  0.6],
                           [1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0],
                           [0.0, 0.1, 0.4,  0.2, 0.0, 0.1, 0.3,  0.2, 0.0, 0.1, 0.4,  0.2, 0.0, 0.1, 0.3,  0.2]]).transpose()

nr.seed(2334)
sims = sim_multinoulli(probs, nsamp)

C_lg = [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1]
B_lg = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
BW_lg = [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1]
CW_lg = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]
M_samps = pd.DataFrame(selec_dist_4(sims, B_samps, C_samps, BW_samps, CW_samps, B_lg, C_lg, BW_lg, CW_lg), columns = ['M'])



## Now concatenate the columns into one data frame
dats = pd.concat([B_samps, C_samps, BW_samps, CW_samps, M_samps], axis = 1)
print(dats.shape)
dats

## Part 1: Parameter Estimation

With the dataset generated you will now estimate the parameters of the graphical model using both maximum likelihood and Bayesian methods. 

As a first step execute the code in the cell below to load the packages you will need.

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import BeliefPropagation
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score, StructureScore

Now, create and execute the code in the cell below to estimate and display the parameters of the CPDs using the graph structure previously shown.

Examine these results and answer the following questions:
1. How many parameters are there in the CPD tables?
2. Given the number of cases, is this MLE problem underdetermined? 

ANS 1:     
ANS 2: 

Next, you will estimate the CPD parameters using the Bayesian estimator. For this first estimate use the following moderately weak and uniform prior distributions (pseudo counts):

- C: {4,4}
- B: {4,4}
- CW: {4,4}
- BW: {4,4}
- M: {2,2,2}

In the cell below create and execute the code to perform Bayes estimation and display the CPD parameters. 

Focus your attention on the M CPD. In terms of extreme values, how does the table computed with the Bayesian method compare to the table computed with MLE? 

ANS: 

Next verify that the independencies of all the variables in your model are correct using the `local_independencies` method. Create and execute the code in the cell below to display the independencies in the CPD. 

**Question:** Is your graph an I-map of the distribution and why?

ANS:

Now you are ready to perform inference on your model. Use the belief propagation method to query the M variable with evidence that the cook is not the murderer. 

Compare this resulting marginal distribution to the marginal distribution you obtained for the same query in the previous homework using the CDP tables provided. How do these distribution differ?

ANS: 

Next, you will estimate the CPD parameters using the Bayesian estimator with a moderately weak but biased prior distribution. For this first estimate use the following moderately weak and uniform prior distributions (pseudo counts):

- C: {2,1}
- B: {1,2}
- CW: {1,2}
- BW: {2,1}
- M: {2,1,2}

In the cell below create and execute the code to perform Bayes estimation and display the CPD parameters. 

Compare the parameters in the M CPD table to the estimate with a uniform prior. How are these different, and is this expected given the change in prior?

ANS: 

Now, use the belief propagation method to query the M variable with evidence that the cook is not the murderer.  

Compare this marginal distribution to the one obtained with the a uniform prior. In general, are the differences what you would expect, and why?

ANS: 

## Part 2

Now you will explore how well the structure of the graph can be estimated. **Keep in mind that the graph used for the simulation is not a tree**. Further, your estimates of graph structure may differ from results obtained in other environment. Answer the questions based on the results you find. 

As a first step, execute the code i the cell below to create a simulated dataset with 2000 cases. 

In [ ]:
## set the sample size
nsamp = 2000

## First the conditionally independent variables
nr.seed(22234)
B_samps = pd.DataFrame(nr.binomial(1, 0.8, nsamp), columns = ['B'])
nr.seed(2355)
C_samps = pd.DataFrame(nr.binomial(1, 0.3, nsamp), columns = ['C'])

## Two variables conditionally depenent on one other
probs = [0.5, 0.1]
nr.seed(2134)
bern = sim_bernoulli(probs, nsamp)
BW_samps = pd.DataFrame(selec_dist_1(bern, B_samps, [0,1]), columns = ['BW'])

probs = [0.5, 0.7]
nr.seed(22234)
bern = sim_bernoulli(probs)
CW_samps = pd.DataFrame(selec_dist_1(bern, C_samps, [0,1]), columns = ['CW'])


probs = np.array([[0.0, 0.7, 0.5,  0.7, 0.0, 0.6, 0.5,  0.6, 0.0, 0.7, 0.5,  0.7, 0.0, 0.6, 0.5,  0.6],
                           [1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0, 1.0, 0.0, 0.0,  0.0],
                           [0.0, 0.1, 0.4,  0.2, 0.0, 0.1, 0.3,  0.2, 0.0, 0.1, 0.4,  0.2, 0.0, 0.1, 0.3,  0.2]]).transpose()

nr.seed(2334)
sims = sim_multinoulli(probs, nsamp)

C_lg = [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1]
B_lg = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
BW_lg = [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1]
CW_lg = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]
M_samps = pd.DataFrame(selec_dist_4(sims, B_samps, C_samps, BW_samps, CW_samps, B_lg, C_lg, BW_lg, CW_lg), columns = ['M'])



## Now concatenate the columns into one data frame
dats_big = pd.concat([B_samps, C_samps, BW_samps, CW_samps, M_samps], axis = 1)
print(dats_big.shape)
dats_big.head(20)

With the dataset simulated, you will now try estimating the model structure. Use the hill climb search algorithm along with the the BIC scoring function to estimate the model structure. Create and execute the code in the cell below to estimate the model structure and display the identified edges. 

Examine these edges. Clearly, this structure is quite different from the structure used for the simulation. Keeping in mind the butler is much more likely to be the murderer compared to the cook, does this structure make sense and why?

ANS: 

How good is this structure fit? To answer this question you will need to compare the BIC score of the graph used for the simulation with the BIC score of the estimated structure. You must create a baseline DAG structure and compare the BIC score to the score of the estimated model. 

In the cell below create the code to define a baseline DAG and compute and display the BIC score of both graphs.

Has the BIC score changed significantly? 

ANS: 

Next, you will apply the K2 score method to the large dataset. In the cell below, create the code to use the hill climbing search with the K2 score to estimate and display the model structure. 

Is this graph structure any different from the one obtained with fewer cases?

ANS: The structure is the same. 

Now, compare the K2 score for the baseline model with the estimated model using the larger dataset for both cases. In the cell below create and execute the code to compute and display these scores. 

How are these K2 scores different? 

ANS: 

In the cell below create and execute the code to display the independencies of the graph structure you have found. 

Finally, compare the BIC and K2 scores of the two models you created with the K2 and BIC score methods on the large dataset. In the cell below create and execute the code to compute and display these 4 scores.  

Are there any substantial differences between the scores of these two models?

ANS: No, they are the same.